In [1]:
import pandas as pd
import re
import string

In [2]:
df = pd.read_csv(r'C:\Users\Harry\Python\words_app\Dictionary in csv\A.csv', encoding='unicode_escape', names=["Raw Data"])

In [3]:
s = list(set(string.ascii_uppercase))
column_names = ['Raw Data']
df = pd.DataFrame(columns = column_names)
# Loop through all spreadsheets and combine them
for i in range(len(s)):
    file_string = 'C:\\Users\\Harry\\Python\\words_app\\Dictionary in csv\\' + s[i] + '.csv'
    latest_df = pd.read_csv(file_string, encoding='unicode_escape', names=["Raw Data"])
    df = df.append(latest_df)

In [4]:
# Split Words, types, and meanings
word_regex = '.+?(?= \()'
type_regex = '\(.+\)'
description_regex = '(?<=\) ).+'
df['Word'] = df['Raw Data'].str.findall(word_regex)
df['Type'] = df['Raw Data'].str.findall(type_regex)
df['Meaning'] = df['Raw Data'].str.findall(description_regex)

df['Word'] = [','.join(map(str, l)) for l in df['Word']]
df['Type'] = [','.join(map(str, l)) for l in df['Type']]
df['Meaning'] = [','.join(map(str, l)) for l in df['Meaning']]

df.drop(columns=['Raw Data'], inplace = True)

In [5]:
# Group types and definitions by words and remove duplicates
df['Type'] = df[['Word','Type','Meaning']].groupby(['Word'])['Type'].transform(lambda x: ','.join(x))
df['Meaning'] = df[['Word','Type','Meaning']].groupby(['Word'])['Meaning'].transform(lambda x: ','.join(x))

df.drop_duplicates(subset=['Word'], keep='first', inplace=True)
df = df.reset_index(drop=True) # Doesn't seem to work

df = df[df['Word'].apply(lambda x: len(x) < 30)]

In [6]:
# Clean up Type column
df['Type'] = df['Type'].str.split(',').apply(set).str.join(',')
df['Type'] = df['Type'].str.findall('\(.*?\)')
df['Type'] = df['Type'].apply(str).str[1:-1]

In [7]:
df

,Word,Type,Meaning
2,A,'(prep.)',"An adjective, commonly called the indefinite a..."
6,Aam,'(n.)',"A Dutch and German measure of liquids, varying..."
7,Aard-vark,'(n.)',"An edentate mammal, of the genus Orycteropus, ..."
9,Aaronic,'(a.)',Alt. of Aaronical
10,Aaronical,'(a.)',"Pertaining to Aaron, the first high priest of ..."
...,...,...,...
114695,Myxomata,'(pl. )',of Myxoma
114696,Myxoma,'(n.)',A tumor made up of a gelatinous tissue resembl...
114697,Myxopod,'(n.)',A rhizopod or moneran. Also used adjectively; ...
114698,Myzontes,'(n. pl.)',The Marsipobranchiata.


In [8]:
# Output to csv for testing
df.to_csv(r'C:\Users\Harry\Python\words_app\test_dictionary.csv', encoding='UTF-8')

In [9]:
# Remove everything with more than 30 characters
# Merge words and meanings
# Save entire dictionary seperate from game dictionary
# Remove words with less than x characters
# Create loop to do this over all csvs in folder, appending them all to one dataframe